# Relazione Finale - Gruppo Dig Data
Componenti Gruppo - Alexandru Pavel, Simone Garzarella

# Indice
TODO: Mettere i numeri sezioni solo qui per chiarezza ulteriore (anche se non serve)
- [Introduzione](#introduction)
    *  [Descrizione Problema](#problem-description)
    *  [Specifiche Software](#sw-specs)
    *  [Specifiche Software](#hw-specs)
- [Analisi Dataset](#data-analysis)
    * [Historical Stock Prices](#hsp)
    * [Historical Stocks](#hs)
- [Job 1](#job1)
    * [MapReduce](#mapreduce1)
    * [Hive](#hive1)
    * [Spark](#spark1)
- [Job 2](#job2)
    * [MapReduce](#mapreduce2)
    * [Hive](#hive2)
    * [Spark](#spark2)
- [Job 3](#job3)
    * [MapReduce](#mapreduce3)
    * [Hive](#hive3)
    * [Spark](#spark3)
- [Grafici Finali](#plots)
    * [Tipo1](#plot1)
    * [Tipo2](#plot2)
    * [Tipo3](#plot3)
- [Conclusioni](#conclusions)

# Introduzione <a name="introduction"></a>
Il dataset "Daily Historical Stock Prices" contiene l'andamento delle azioni sulla borsa di New York (NYSE e NASDAQ) dal 1970 al 2018. 

Due file CSV compongono il dataset:
- historical_stock_prices.csv
- historical_stocks.csv

Il primo contiene i valori dei prezzi e volumi che variano nel tempo per ogni ticker. Il secondo i dati relativi ad ogni ticker, come il settore e l'exchange in cui è quotato.

## Descrizione Problema <a name="problem-description"></a>
Dopo una fase iniziale di analisi e processamento di dati si vogliono eseguire 3 job (descritti nel dettaglio più avanti) con le diverse tecnologie affrontate nel corso (Hadoop, Hive e Apache Spark).

## Specifiche Software <a name="sw-specs"></a>
Serve? Python, Hadoop, Hive, Spark con versioni

## Specifiche Hardware <a name="hw-specs"></a>
I test sono stati eseguiti in locale e su cluster con macchine con queste caratteristiche.
### Locale
Ubuntu 20.04, CPU i5 2.5GHZ, 8GB Ram e 256GB SSD
### Cluster
Amazon Web Services EMR su Cluster con 1 Master Node e 5 Data Node. Tutte le istanze sono m5.xlarge con 16GB RAM, 4 vCPU e 50GB SSD.

# Analisi Dataset <a name="data-analysis"></a>

Di seguito vengono analizzati i due file del dataset per individuare eventuali preprocessamenti da effettuare. Inoltre viene anche descritto il processo per creare dataset più piccoli o grandi (con sampling) per effettuare i successivi test.

In [4]:
import pandas as pd

## Historical Stock Prices <a name="hsp"></a>

I campi di questo dataset sono:

- `ticker`: simbolo univoco dell’azione (https://en.wikipedia.org/wiki/Ticker_symbol)
- `open`: prezzo di apertura
- `close`: prezzo di chiusura
- `adj_close`: prezzo di chiusura “modificato”
- `lowThe`: prezzo minimo
- `highThe`: prezzo massimo
- `volume`: numero di transazioni
- `date`: data nel formato aaaa-mm-gg

In [10]:
stock_prices = pd.read_csv('dataset/historical_stock_prices.csv')

In [11]:
stock_prices

,ticker,open,close,adj_close,low,high,volume,date
0,AHH,11.50,11.58,8.493155,11.25,11.68,4633900,2013-05-08
1,AHH,11.66,11.55,8.471151,11.50,11.66,275800,2013-05-09
2,AHH,11.55,11.60,8.507822,11.50,11.60,277100,2013-05-10
3,AHH,11.63,11.65,8.544494,11.55,11.65,147400,2013-05-13
4,AHH,11.60,11.53,8.456484,11.50,11.60,184100,2013-05-14
...,...,...,...,...,...,...,...,...
20973884,NZF,14.60,14.59,14.590000,14.58,14.62,137500,2018-08-20
20973885,NZF,14.60,14.58,14.580000,14.57,14.61,151200,2018-08-21
20973886,NZF,14.58,14.59,14.590000,14.57,14.63,185400,2018-08-22
20973887,NZF,14.60,14.57,14.570000,14.57,14.64,135600,2018-08-23


Ci sono ~21milioni di record per questo file

In [12]:
stock_prices.isna().sum()

ticker       0
open         0
close        0
adj_close    0
low          0
high         0
volume       0
date         0
dtype: int64

Non sono presenti valori nulli per nessuna delle colonne

In [13]:
stock_prices.nunique()

ticker          5685
open          807688
close         835181
adj_close    9235753
low           815237
high          821044
volume        385849
date           12274
dtype: int64

In totale ci sono 5685 `ticker` univoci nel dataset

In [14]:
stock_prices[stock_prices.duplicated(subset=['ticker','date'])].shape

(0, 8)

Non ci sono record distinti con valori duplicati di (ticker, data)

### Creazione di dataset di varie dimensioni

Sono stati generati dataset di dimensioni (approssimativamente) di 256/512/1024MB e ~4GB, oltre al dataset originale che ha dimensioni ~2GB.

I file generati (con relativa dimensione precisa) hanno nome historical_stock_prices[size].csv

- historical_stock_prices`256`.csv &ensp;&ensp;(239.75MB)
- historical_stock_prices`512`.csv &ensp;&ensp;(479.51MB)
- historical_stock_prices`1024`.csv &ensp;(959.03MB)
- historical_stock_prices.csv &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;(1909.97MB)
- historical_stock_prices`4096`.csv &ensp;(3835.92MB)

La scelta dei record da includere è effettuata con un sampling randomico (con un seed preimpostato, per la ripetibiltà)

```python
def sample_all_sizes(historical_stock_prices_df):
    for size in [0.125, 0.25, 0.5, 2]:
        sample_n_rows = dataset_row_count * size
        sampled_df = dataset.sample(sample_n_rows)
        filename = 'dataset/historical_stock_prices[SIZE].csv')
        sampled_df.to_csv(filename)

```

## Historical Stocks <a name="hs"></a>

Il dataset con le informazioni sui ticker è così strutturato:

- `ticker`: simbolo dell’azione
- `exchange`: NYSE o NASDAQ
- `name`: nome dell’azienda
- `sector`: settore dell’azienda
- `industry`: industria di riferimento per l’azienda

In [15]:
stocks = pd.read_csv("dataset/historical_stocks.csv")

In [16]:
stocks

,ticker,exchange,name,sector,industry
0,PIH,NASDAQ,"1347 PROPERTY INSURANCE HOLDINGS, INC.",FINANCE,PROPERTY-CASUALTY INSURERS
1,PIHPP,NASDAQ,"1347 PROPERTY INSURANCE HOLDINGS, INC.",FINANCE,PROPERTY-CASUALTY INSURERS
2,TURN,NASDAQ,180 DEGREE CAPITAL CORP.,FINANCE,FINANCE/INVESTORS SERVICES
3,FLWS,NASDAQ,"1-800 FLOWERS.COM, INC.",CONSUMER SERVICES,OTHER SPECIALTY STORES
4,FCCY,NASDAQ,1ST CONSTITUTION BANCORP (NJ),FINANCE,SAVINGS INSTITUTIONS
...,...,...,...,...,...
6455,ZOES,NYSE,"ZOE&#39;S KITCHEN, INC.",CONSUMER SERVICES,RESTAURANTS
6456,ZTS,NYSE,ZOETIS INC.,HEALTH CARE,MAJOR PHARMACEUTICALS
6457,ZTO,NYSE,ZTO EXPRESS (CAYMAN) INC.,TRANSPORTATION,TRUCKING FREIGHT/COURIER SERVICES
6458,ZUO,NYSE,"ZUORA, INC.",TECHNOLOGY,COMPUTER SOFTWARE: PREPACKAGED SOFTWARE


In [18]:
stocks.nunique()

ticker      6460
exchange       2
name        5462
sector        13
industry     136
dtype: int64

Sono presenti 6460 `ticker` univoci, come il numero di righe del dataset. Il ticker può essere considerato una chiave di questo dataset, il nome dell'azienda `name` invece no, ha delle ripetizioni.

In [17]:
stocks[stocks.duplicated(subset=['name'])].shape

(998, 5)

In particolare sono presenti 998 nomi di azienda duplicati. Nel resto del progetto non si considererà questo campo per identificare record (in particolare per il job3).

In [20]:
stocks['sector'].unique()

array(['FINANCE', 'CONSUMER SERVICES', 'TECHNOLOGY', 'PUBLIC UTILITIES',
       'CAPITAL GOODS', 'BASIC INDUSTRIES', 'HEALTH CARE',
       'CONSUMER DURABLES', nan, 'ENERGY', 'MISCELLANEOUS', 'SECTOR',
       'TRANSPORTATION', 'CONSUMER NON-DURABLES'], dtype=object)

Visualizzando i possibili valori di `sector` si può notare la presenza di un valore nullo.

In [19]:
stocks.isna().sum()

ticker         0
exchange       0
name           0
sector      1440
industry    1440
dtype: int64

Il campo `sector` presenta 1440 valori nulli, che vengono eliminati durante il preprocessing di questo dataset.

In [21]:
stocks_clean = stocks.loc[(stocks['sector'].notna())]

In [23]:
stocks_clean.shape

(5020, 5)

Il dataset pulito dai valori nulli del campo `sector` ha 5020 record. Verrà salvato come `historical_stocks_clean.csv`

```python
stocks_clean.to_csv('dataset/historical_stocks_clean.csv')
```

# Job 1 <a name="job1"></a>
The first paragraph text

## MapReduce <a name="mapreduce1"></a>
This is a sub paragraph, formatted in heading 3 style

## Hive <a name="hive1"></a>
This is a sub paragraph, formatted in heading 3 style

## Spark <a name="spark1"></a>
This is a sub paragraph, formatted in heading 3 style

# Job 2 <a name="job2"></a>
The first paragraph text

## MapReduce <a name="mapreduce2"></a>
This is a sub paragraph, formatted in heading 3 style

## Hive <a name="hive2"></a>
This is a sub paragraph, formatted in heading 3 style

## Spark <a name="spark2"></a>
This is a sub paragraph, formatted in heading 3 style

# Job 3 <a name="job3"></a>
The first paragraph text

## MapReduce <a name="mapreduce3"></a>
This is a sub paragraph, formatted in heading 3 style

## Hive <a name="hive3"></a>
This is a sub paragraph, formatted in heading 3 style

## Spark <a name="spark3"></a>
This is a sub paragraph, formatted in heading 3 style

# Grafici <a name="plots"></a>
The first paragraph text

## MapReduce <a name="plot1"></a>
This is a sub paragraph, formatted in heading 3 style

## Hive <a name="plot2"></a>
This is a sub paragraph, formatted in heading 3 style

## Spark <a name="plot3"></a>
This is a sub paragraph, formatted in heading 3 style

# Conclusioni <a name="conclusions"></a>
The first paragraph text
